In [224]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import requests
from input_features_for_predict import input_app_info

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [260]:
def get_train_valid(train, predict):
    
    
    train.drop(['r_five_players_id', 'd_five_players_id'], 1, inplace=True)
    train.drop('r_org', 1, inplace=True)
    train.drop('d_org', 1, inplace=True)
    train.drop('duration', 1, inplace=True)
    

    predict.drop(['r_five_players_id', 'd_five_players_id'], 1, inplace=True)
    predict.drop('r_org', 1, inplace=True)
    predict.drop('d_org', 1, inplace=True)
    
    
    columns = []
    columns.extend([f'r_{i}_player_id' for i in range(1, 6)])
    columns.extend([f'd_{i}_player_id' for i in range(1, 6)])
    train.drop(columns, 1, inplace=True)
    predict.drop(columns, 1, inplace=True)
    
    heroes_unique_values = sorted(list(set(list(train[[f'r_{i}' for i in range(1, 6)]].values.ravel()) + list(train[[f'd_{i}' for i in range(1, 6)]].values.ravel()))))
    dummy_heroes_column_names = [f'hero_{i}' for i in heroes_unique_values]
    dummy_heroes_shape = (train.shape[0], len(dummy_heroes_column_names))
    dummy_heroes = pd.DataFrame(np.zeros(dummy_heroes_shape), columns=dummy_heroes_column_names).astype("int8")

    for feature in [f'r_{i}' for i in range(1, 6)]:
        encoded_feature = pd.get_dummies(train[feature], prefix="hero")
        dummy_heroes[encoded_feature.columns] += encoded_feature

    for feature in [f'd_{i}' for i in range(1, 6)]:
        encoded_feature = pd.get_dummies(train[feature], prefix="hero")
        dummy_heroes[encoded_feature.columns] -= encoded_feature
    train = pd.concat([train, dummy_heroes], axis=1)
    train.drop([f'r_{i}' for i in range(1, 6)], 1, inplace=True)
    train.drop([f'd_{i}' for i in range(1, 6)], 1, inplace=True)
    heroes_columns = encoded_feature.columns

    dummy_heroes_shape = (predict.shape[0], len(dummy_heroes_column_names))
    dummy_heroes = pd.DataFrame(np.zeros(dummy_heroes_shape), columns=dummy_heroes_column_names).astype("int8")
    for feature in [f'r_{i}' for i in range(1, 6)]:
        encoded_feature = pd.get_dummies(predict[feature], prefix="hero")
        dummy_heroes[encoded_feature.columns] += encoded_feature

    for feature in [f'd_{i}' for i in range(1, 6)]:
        encoded_feature = pd.get_dummies(predict[feature], prefix="hero")
        dummy_heroes[encoded_feature.columns] -= encoded_feature
    predict = pd.concat([predict, dummy_heroes], axis=1)
    predict.drop([f'r_{i}' for i in range(1, 6)], 1, inplace=True)
    predict.drop([f'd_{i}' for i in range(1, 6)], 1, inplace=True)
    
    train['r_win'] = train['r_win'].astype('int8')
    
    winrate_columns = [col for col in train.columns.values if 'winrate' in col.split('_')]
    columns_to_not_scale = list(heroes_columns.values) + winrate_columns + ['r_win']
    columns_to_scale = list(set(train.columns) - set(columns_to_not_scale))
    
    scaler = MinMaxScaler()
    scaled_columns_train = scaler.fit_transform(train[columns_to_scale])
    scaled_columns_predict = scaler.transform(predict[columns_to_scale])
    for i, col in enumerate(columns_to_scale):
        train[col] = scaled_columns_train[:, i]
        predict[col] = scaled_columns_predict[:, i]
        
        
    x_train, y_train = train.drop('r_win', 1), train['r_win'].values
    
    return x_train, y_train, predict

In [190]:
live_games = requests.get('https://datdota.com/api/livegames').json()['data']['matches']

In [192]:
match = requests.get(f'https://datdota.com/api/livegames/{6457276195}?').json()['data']

In [64]:
match_players_info = []
game = {}
game['r_org'] = match['dire_team']['team_id']
game['d_org'] = match['radiant_team']['team_id']

accounts_id = {}
i = 0
for player in match['players']:
    if player['team'] in [0, 1]:
        i += 1
        game[f'player_id_{i}'] = player['account_id']
        game[f'player_name_{i}'] = player['name']
        game[f'player_hero_{i}'] = player['hero_id']
    
    
match_players_info.append(game)
match_players_info = pd.DataFrame(match_players_info, columns=list(game.keys()))

In [235]:
data = pd.DataFrame()
for col in ['r_org', 'd_org','r_1', 'r_1_player_id', 'd_1',
       'd_1_player_id', 'd_2', 'd_2_player_id', 'r_2', 'r_2_player_id', 'd_3',
       'd_3_player_id', 'r_3', 'r_3_player_id', 'r_4', 'r_4_player_id', 'd_4',
       'd_4_player_id', 'r_5', 'r_5_player_id', 'd_5', 'd_5_player_id']:
    data.loc[0, col] = 0

In [236]:
data['r_org'] = 2586976
data['d_org'] = 8572181

data['r_1'] = 106
data['r_1_player_id'] = 93618577

data['d_1'] = 121
data['d_1_player_id'] = 76904792

data['d_2'] = 107
data['d_2_player_id'] = 86700461

data['r_2'] = 90
data['r_2_player_id'] = 401792574

data['d_3'] = 88
data['d_3_player_id'] = 152859296

data['r_3'] = 27
data['r_3_player_id'] = 117483894

data['r_4'] = 95
data['r_4_player_id'] = 167976729

data['d_4'] = 55
data['d_4_player_id'] = 91369376

data['r_5'] = 98
data['r_5_player_id'] = 183719386

data['d_5'] = 94
data['d_5_player_id'] = 181716137

In [244]:
result = data.copy()
game_with_features = input_app_info(result)

In [261]:
train = pd.read_csv('train.csv')
predict = game_with_features.copy()
x_train, y_train, x_valid = get_train_valid(train, predict)

In [259]:
forest = RandomForestClassifier(n_estimators=400, n_jobs=-1, random_state=17, max_depth=8, max_features='sqrt')
forest.fit(x_train, y_train)
result = forest.predict(x_valid).astype('bool')[0]
print(f'radiant win --> {result}')

radiant win False
